# CMDPにおける安全な探索とは？


# 準備
CMDPはいつもの$\left\{\mathcal{S}, \mathcal{A}, P, \rho,\left\{r_{i}\right\}_{i=0}^{m}, \gamma\right\}$で定義されます。$r_{0}: \mathcal{S} \times \mathcal{A} \rightarrow[0,1]$は報酬関数であり、$r_{i}: \mathcal{S} \times \mathcal{A} \rightarrow[-1,1]$は$i \in[m]$に対するユーティリティ関数です。



**学びポイント**
定常方策のみを最適化の対象にするのはOKです。Altmanによると、CMDPにおける任意の最適方策の集合は定常方策が含まれているからです。
* 一方、期待値制約以外の制約の場合はコレは成り立つのだろうか？
パラメータ化する場合、それによって表現できる方策集合は確率的方策集合全体をカバーするか分からないので、良い感じに最適方策に近似できる方策を見つける必要があるみたいです。

**安全な探索とは？**
安全な探索をこの論文では以下のように定義しています。

列$\left\{\theta_{t}\right\}_{t=0}^{T}$を提供するアルゴリズムが与えられたとき、$\forall t \in\{0, \ldots, T\}$に対して$\min _{i \in[m]} V_{i}^{\theta_{t}}(\rho) \geq 0$が成り立つ場合、そのアルゴリズムは安全な探索を保証する。

要は、制約関数のワーストケース取ってきても制約満たしているということす。


**安全な探索のためのアプローチ：対数バリア**
この研究では、
$$
\begin{equation*}
\max _{\theta} V_{0}^{\theta}(\rho) \quad \text{s.t.} \quad V_{i}^{\theta}(\rho) \geq 0, \quad i \in[m] \tag{RL-P}
\end{equation*}
$$
という最適化問題について、対数バリアという方法を使って以下のような制約なし最適化問題に書き換えているようです。
$$
B_{\eta}^{\theta}(\rho):=V_{0}^{\theta}(\rho)+\eta \sum_{i=1}^{m} \log V_{i}^{\theta}(\rho)
$$
バリア関数は実行可能領域でのみ定義されるため、制約を違反することなく最適化が行えるようです。なぜなら$\log(\cdot)$は0より大きい場合のみ定義できる関数だからです。パラメータのアップデートは以下の通りです。
$$
\begin{equation*}
\theta_{t+1}=\theta_{t}+\gamma_{t} \hat{\nabla}_{\theta} B_{\eta}^{\theta}(\rho) \tag{1}
\end{equation*}
$$
$\hat{\nabla}_{\theta} B_{\eta}^{\theta}(\rho)$は真の勾配$\nabla_{\theta} B_{\eta}^{\theta}(\rho)$の推定量です。それは以下で計算できてしまいます。
$$
\begin{equation*}
\nabla_{\theta} B_{\eta}^{\theta}(\rho)=\nabla V_{0}^{\theta}(\rho)+\eta \sum_{i=1}^{m} \frac{\nabla V_{i}^{\theta}(\rho)}{V_{i}^{\theta}(\rho)} \tag{2}
\end{equation*}
$$
**$\hat{\nabla}_{\theta} B_{\eta}^{\theta}(\rho)$をどうやって計算するか？**
方策の辿った軌跡から推定するのが定石です。軌跡というか軌道は以下のように示されます。
$$\tau_{j}:= \left(s_{t}^{j}, a_{t}^{j},\left\{r_{i}\left(s_{t}^{j}, a_{t}^{j}\right)\right\}_{i=0}^{m}\right)_{t=0}^{H-1}$$
コレを使って$\hat{V}_{i}^{\theta}(\rho)$と$\nabla_{\theta} V_{i}^{\theta}(\rho)$を以下のように推定します。
$$
\hat{V}_{i}^{\theta}(\rho):=\frac{1}{n} \sum_{j=1}^{n} \sum_{t=0}^{H-1} \gamma^{t} r_{i}\left(s_{t}^{j}, a_{t}^{j}\right)
$$
$$
\hat{\nabla}_{\theta} V_{i}^{\theta}(\rho):=\frac{1}{n} \sum_{j=1}^{n} \sum_{t=0}^{H-1} \sum_{t^{\prime}=0}^{t} \gamma^{t} r_{i}\left(s_{t}^{j}, a_{t}^{j}\right) \nabla_{\theta} \log \pi_{\theta}\left(a_{t^{\prime}}^{j} \mid s_{t^{\prime}}^{j}\right)
$$

次に仮定として、$\nabla_{\theta} \log \pi_{\theta}\left(a_{t^{\prime}}^{j} \mid s_{t^{\prime}}^{j}\right)$がいくつかの良い感じの性質を持っていることを示します。

**仮定3.1**
$\log \pi_{\theta}(a \mid s)$の勾配とヘッセ行列(二次勾配の行列)は有界です。
定数$M_{g}, M_{h}>0$が存在し、すべての$\theta \in \Theta$に対して$\left\|\nabla_{\theta} \log \pi_{\theta}(a \mid s)\right\| \leq M_{g}$かつ$\left\|\nabla_{\theta}^{2} \log \pi_{\theta}(a \mid s)\right\| \leq M_{h}$が成り立ちます。

**命題3.3：仮定3.1が成立すると、成り立つ良い感じの性質**
以下の性質が$\forall i \in\{0, \ldots, m\}$および$\forall \theta \in \Theta$に対して成り立つ。

1. $V_{i}^{\theta}(\rho)$は$L$-リプシッツ連続かつ$M$-平滑($L:=\frac{M_{g}}{(1-\gamma)^{2}}$、$M:=\frac{M_{g}^{2}+M_{h}}{(1-\gamma)^{2}}$)
* リプシッツ連続なので一様収束を議論しやすくなります
* 平滑なので必ず1回微分可能です

2. 以下が成り立ちます。
$$
\begin{aligned}
& \left|V_{i}^{\theta}(\rho)-\mathbb{E}\left[\hat{V}_{i}^{\theta}(\rho)\right]\right| \leq b^{0}(H), \\
& \left\|\nabla_{\theta} V_{i}^{\theta}(\rho)-\mathbb{E}\left[\hat{\nabla}_{\theta} V_{i}^{\theta}(\rho)\right]\right\| \leq b^{1}(H) .
\end{aligned}
$$
　　　　　　　　　　　　　　　　　　　　　　($b^{0}(H):=\frac{\gamma^{H}}{1-\gamma}$、$b^{1}(H):=\frac{M_{g} \gamma^{H}}{1-\gamma} \sqrt{\frac{1}{1-\gamma}+H}$)
* 真の値と期待推定値の誤差上界を出しています

2. 任意の$\delta \in(0,1)$に対して、
$$
\mathbb{P}\left(\left|\hat{V}_{i}^{\theta}(\rho)-\mathbb{E}\left[\hat{V}_{i}^{\theta}(\rho)\right]\right| \leq \sigma^{0}(n) \sqrt{\ln \frac{2}{\delta}}\right) \geq 1-\delta
$$
　　　　　　　　　　　　　　　　　　　　　　($\sigma^{0}(n):=\frac{\sqrt{2}}{\sqrt{n}(1-\gamma)}$、$\sigma^{1}(n):=\frac{2 \sqrt{2} M_{g}}{\sqrt{n}(1-\gamma)^{\frac{3}{2}}}$)
3. $n \geq 8 \ln \frac{e^{\frac{1}{4}}}{\delta}$の場合、
$$
\mathbb{P}\left(\left\|\hat{\nabla}_{\theta} V_{i}^{\theta}(\rho)-\mathbb{E}\left[\hat{\nabla}_{\theta} V_{i}^{\theta}(\rho)\right]\right\| \leq \sigma^{1}(n) \sqrt{\ln \frac{e^{\frac{1}{4}}}{\delta}}\right) \geq 1-\delta
$$
* PAC保証です

**補題3.4：仮定3.1が成り立つときに成立するサンプル複雑度的なやつ**
信頼水準$\delta \in(0,1]$と精度$\varepsilon>0$を固定すると、以下が成立
$$
\mathbb{P}\left(\left\|\hat{\nabla}_{\theta} B_{\eta}^{\theta}(\rho)-\nabla_{\theta} B_{\eta}^{\theta}(\rho)\right\| \leq \varepsilon\right) \geq 1-\delta
$$
* $n=\mathcal{O}\left(\varepsilon^{-2}(1-\gamma)^{-6}\left(\min _{i \in[m]} V_{i}^{\theta}(\rho)\right)^{-4} \eta^{2} \ln \delta^{-1}\right)$
* $H=\mathcal{O}\left(\ln \left(\varepsilon \min _{i \in[m]} V_{i}^{\theta}(\rho)\right)^{-1}\right)$


# アルゴリズム
 
 アルゴリズム：LB-SGD
    入力: 平滑性パラメータ $M=\frac{M_{g}^{2}+M_{h}}{(1-\gamma)^{2}}$,
    バッチサイズ $n$, ホライズン $H$, 反復回数 $T$,  $\delta \in(0,1), \eta>0$.
    for $t=0,1, \ldots, T-1$ do
         $\hat{V}_{i}^{\theta}(\rho):=\frac{1}{n} \sum_{j=1}^{n} \sum_{t=0}^{H-1} \gamma^{t} r_{i}\left(s_{t}^{j}, a_{t}^{j}\right)$と$\hat{\nabla}_{\theta} V_{i}^{\theta}(\rho):=\frac{1}{n} \sum_{j=1}^{n} \sum_{t=0}^{H-1} \sum_{t^{\prime}=0}^{t} \gamma^{t} r_{i}\left(s_{t}^{j}, a_{t}^{j}\right) \nabla_{\theta} \log \pi_{\theta}\left(a_{t^{\prime}}^{j} \mid s_{t^{\prime}}^{j}\right)$を用いて$\hat{V}_{i}^{\theta_{t}}(\rho)$ と $\hat{\nabla}_{\theta} V_{i}^{\theta_{t}}(\rho)$ を計算
        $\hat{\nabla}_{\theta} B_{\eta}^{\theta}(\rho):=\hat{\nabla}_{\theta} V_{0}^{\theta}(\rho)+\eta \sum_{i=1}^{m} \frac{\hat{\nabla}_{\theta} V_{i}^{\theta}(\rho)}{\hat{V}_{i}^{\theta}(\rho)}$を用いて $\hat{\nabla}_{\theta} B_{\eta}^{\theta_{t}}(\rho)$ を計算
        if $\left\|\hat{\nabla}_{\theta} B_{\eta}^{\theta_{t}}(\rho)\right\| \leq \frac{\eta}{2}$ then
            中断、$\theta_{\text {break }}$ を返す
        end if
        $\theta_{t+1}=\theta_{t}+\gamma_{t} \hat{\nabla}_{\theta} B_{\eta}^{\theta_{t}}(\rho)$を計算(なお、$\gamma_{t}=\min \left\{\frac{1}{\hat{M}_{t}}, \min _{i \in[m]}\left\{\frac{\underline{\alpha}_{i}(t)}{\sqrt{M \underline{\alpha}_{i}(t)}+2\left|\bar{\beta}_{i}(t)\right|}\right\} \frac{1}{\left\|\hat{\nabla}_{x} B_{\eta}^{\theta_{t}}(\rho)\right\|}\right\}$)
    end for
    $\theta_{\text {break }}$ または $\theta_{T}$ のいずれかである $\theta_{\text {out }}$ を返す

**定理3.5：LB-SGDは高確率で安全な探索を行い、高確率で準最適方策を獲得出るという性質をもつ**

# 安全な探索の証明
本ノートのメインです。収束解析の証明は取り扱いません。

## 仮定
**証明のための仮定**
CMDPにおいて定番のスレーター条件を最初に仮定します。これは、制約を満たす実行可能な解が存在していることを示します。特に、今回は開始時点で方策が必ず制約を満たしている必要があるので、以下のように仮定します。

**仮定4.1:**
既知の開始点$\theta_{0} \in \Theta$と$\nu_{s}>0$が存在し、$V_{i}^{\theta_{0}}(\rho) \geq \nu_{s}, \forall i \in[m]$を満たします

つぎの仮定はかなり重要です。

**仮定4.2：拡張MFCQ仮定**
$0<\nu_{\text {emf }} \leq \nu_{s}$と$\ell>0$が存在し、任意の$\theta \in \Theta$に対して、$\left\|s_{\theta}\right\|=1$を満たす方向$s_{\theta} \in \mathbb{R}^{d}$が存在し、すべての$i \in \mathbf{B}_{\nu_{\text {emf }}}(\theta)$に対して
$$\left\langle s_{\theta}, \nabla V_{i}^{\theta}(\rho)\right\rangle>\ell$$を満たします。
* 要は、反復点というかパラメータ$\theta$が制約の境界にかなり近い、つまり$\nu_{\text {emf }}$なら、ある良い感じの方向$s_{\theta} \in \mathbb{R}^{d}$して、その方向に従ったら制約関数が増加していくということです。ある程度ちゃんと制約境界の内側に押し戻す力が存在していることを示しています。

## 命題4.3：高確率で安全な探索を達成
仮定3.1、4.1、および4.2が成り立つとします。
$\delta \in(0,1]$を固定します。$\eta \leq \nu_{e m f}, n=\mathcal{O}\left(\eta^{-4} \ell^{-4 m}(1-\gamma)^{-6-8 m} \ln \delta^{-1}\right)$および$H= \mathcal{O}\left(\ln (\ell \eta)^{-1}\right)$とすると、

$$
\mathbb{P}\left\{\forall t \in\{0, \ldots, T\}, \min _{i \in[m]} V_{i}^{\theta_{t}}(\rho) \geq c \eta\right\} \geq 1-m T \delta
$$

が成り立ちます。なお、$c:=\left(\frac{\ell(1-\gamma)^{2}}{4 M_{g}\left(1+\frac{4 m}{3}\right)}\right)^{m}$です。
* 高確率で、安全な探索が達成されることを示しています。

## 命題の証明

$$
    \mathcal{A} := \left\{\forall t\le T, \ \min_i V_i^{\theta_t}(\rho)\ge c\eta\right\},\quad
    \mathcal{B} := \left\{\forall t\le T, \ \gamma_t\ge C\eta\right\},\quad
    \mathcal{C} := \left\{\forall t\le T, \ |\Delta_t|\ge \tfrac{\eta}{4}\right\}.
    $$
とします。補題D.5→D.4→命題　の順になっています。

### 補題 D.5 
($\min_i \alpha_i(t)\le \frac{\ell\eta}{L(1+\frac{4m}{3})}$) のとき、
任意の ($\mathbf{B}$)（$\mathbf{B}_{\eta}(\theta_t)\subset \mathbf{B}$）に対して
$$
\mathbb{P}\left\{\prod_{i\in\mathbf{B}}\alpha_i(t+1) \ge \prod_{i\in\mathbf{B}}\alpha_i(t)\right\} \ge 1-\delta
$$
が成り立ちます。

### 補題 D.5 の証明
関数
$$
g_t(\theta) := \eta\sum_{i\in\mathbf{B}_{\eta}(\theta_t)}\log V_i^\theta(\rho)
$$
は、局所M-平滑（定数 ($M_t$)）とします。よって
$$
g_t(\theta_{t+1}) \ge g_t(\theta_t)
+ \langle \nabla g_t(\theta_t), \theta_{t+1}-\theta_t \rangle
- \frac{M_t}{2}|\theta_{t+1}-\theta_t|^2.
$$
ここで
$$
\nabla g_t(\theta_t) = \eta\sum_{i\in\mathbf{B}_{\eta}(\theta_t)}\frac{\nabla_{\theta} V_i^{\theta_t}(\rho)}{\alpha_i(t)}, \quad
\theta_{t+1}-\theta_t = \gamma_t \hat\nabla_\theta B_\eta^{\theta_t}(\rho).
$$
代入すると
$$
\begin{aligned}
& \eta\sum_{i\in\mathbf{B}_{\eta}(\theta_t)}\log \alpha_i(t+1)-\eta\sum_{i\in\mathbf{B}_{\eta}(\theta_t)}\log \alpha_i(t) \\
&\ge \gamma_t\left\langle \eta\sum_{i\in\mathbf{B}_{\eta}(\theta_t)}\frac{\nabla_{\theta} V_i^{\theta_t}(\rho)}{\alpha_i(t)}, \hat\nabla_\theta B_\eta^{\theta_t}(\rho)\right\rangle
- \frac{M_t\gamma_t^2}{2}|\hat\nabla_\theta B_\eta^{\theta_t}(\rho)|^2.
\end{aligned}
$$
ステップサイズの構成より ($\mathbb{P}(\gamma_t\le 1/M_t)\ge 1-\delta$) なので
$$
\ge \gamma_t\left(
\left\langle \eta \sum_{i\in\mathbf{B}_{\eta}}\frac{\nabla V_i}{\alpha_i}, \hat\nabla B_{\eta}\right\rangle
- \frac{1}{2}|\hat\nabla B_\eta|^2\right).
$$
ここで
$$
D_1 := \sum_{i\in\mathbf{B}_{\eta}(\theta_t)}\frac{\nabla_{\theta} V_i}{\alpha_i(t)}, \qquad
D_2 := \frac{\hat\nabla_\theta B_\eta}{\eta}-\sum_{i\in\mathbf{B}_{\eta}(\theta_t)}\frac{\nabla_{\theta} V_i}{\alpha_i(t)}.
$$
とおくと
$$
\hat\nabla_\theta B_\eta = \eta(D_1+D_2).
$$
よって上式は
$$
\frac{\gamma_t\eta^2}{2}\left(2\langle D_1, D_1+D_2\rangle-|D_1+D_2|^2\right)
= \frac{\gamma_t\eta^2}{2}\left(|D_1|^2-|D_2|^2\right),
$$
すなわち
$$
\boxed{
\eta \sum_{i\in\mathbf{B}_{\eta}} \log\alpha_i(t+1)-\eta \sum_{i\in\mathbf{B}_\eta} \log\alpha_i(t)
\ge
\frac{\gamma_t\eta^2}{2}\left(|D_1|^2-|D_2|^2\right)
}.
$$
ここでは恒等式($2\langle a,b\rangle-|b|^2=|a|^2-|a-b|^2$)を ($a=D_1, b=D_1+D_2$) に適用しています。

次に、拡張 MFCQ より単位ベクトル ($s_\theta$) があって
($\langle s_\theta,\nabla V_i\rangle\ge \ell$)（$i\in \mathbf{B}_{\eta}(\theta_t)$, $\eta\le \nu_{\rm emf}$）よって
$$
|D_1|
\ge \langle D_1,s_\theta\rangle
= \sum_{i\in\mathbf{B}_{\eta}}\frac{\langle s_{\theta},\nabla V_i\rangle}{\alpha_i(t)}
\ge \sum_{i\in\mathbf{B}_{\eta}}\frac{\ell}{\alpha_i(t)}.
$$
いま仮定より ($\min_i \alpha_i(t)\le \frac{\ell\eta}{L(1+\frac{4m}{3})}$) なので、少なくとも一つの指標 ($i^\star$) に対して
($\ell/\alpha_{i^\star}(t) \ge L(1+\frac{4m}{3})/\eta$)
が成り立ちます。従って
$$
\boxed{ |D_1| \ge \frac{L(1+\frac{4m}{3})}{\eta} }.
$$

次に、定義から
$$
\begin{align*}
|D_2|
&= \left|
\tfrac{\hat\nabla V_0}{\eta}
+\sum_{i\notin\mathbf{B}_{\eta}}\tfrac{\hat\nabla V_i}{\hat\alpha_i(t)}
+\sum_{i\in\mathbf{B}_{\eta}}\tfrac{\hat\nabla V_i}{\hat\alpha_i(t)}
-\sum_{i\in\mathbf{B}_{\eta}}\tfrac{\nabla V_i}{\alpha_i(t)}
\right| \tag{18–19} \\[2mm]
&\le \frac{L}{\eta}\left(1+\frac{4}{3}\left(m-|\mathbf{B}_{\eta}|\right)\right)
+\left|\sum_{i\in\mathbf{B}_{\eta}}\left(
\tfrac{\hat\nabla V_i}{\hat\alpha_i}-\tfrac{\hat\nabla V_i}{\alpha_i}
+\tfrac{\hat\nabla V_i}{\alpha_i}-\tfrac{\nabla V_i}{\alpha_i}
\right)\right| \tag{20} \\[1mm]
&\le \sum_{i\in\mathbf{B}_{\eta}}\left(
\tfrac{|\hat\nabla V_i-\nabla V_i|}{\alpha_i}
+\left|\tfrac{1}{\hat\alpha_i}-\tfrac{1}{\alpha_i}\right| |\hat\nabla V_i|
\right)
+\frac{L}{\eta}\left(1+\frac{4}{3}\left(m-|\mathbf{B}_{\eta}|\right)\right). \tag{21}
\end{align*}
$$
ここで
($|\hat\nabla V_i-\nabla V_i|\le \sigma^1(n)\sqrt{\ln\frac{e^{1/4}}{\delta}}+b^1(H)$),($|\hat\alpha_i-\alpha_i|\le \sigma^0(n)\sqrt{\ln\frac{2}{\delta}}+b^0(H)$),
($|\hat\nabla V_i|\le L$)

命題3.3を使い、さらに ($i\notin\mathbf{B}_{\eta}$) には ($\hat\alpha_i(t)\ge \tfrac{3}{4}\eta$)を用いて
$$
\le \sum_{i\in\mathbf{B}_{\eta}}\left(
\tfrac{\sigma^1\sqrt{\ln\frac{e^{1/4}}{\delta}}+b^1}{\alpha_i}
+L\cdot \tfrac{\sigma^0\sqrt{\ln\frac{2}{\delta}}+b^0}{\hat\alpha_i \alpha_i}
\right)
+\frac{L}{\eta}\left(1+\frac{4}{3}\left(m-|\mathbf{B}_{\eta}|\right)\right). \tag{22}
$$
さらに ($\hat\alpha_i\ge \tfrac{3}{4}\alpha_i$) から
$$
\boxed{
|D_2|
\le \sum_{i\in\mathbf{B}_{\eta}}\left(
\tfrac{\sigma^1\sqrt{\ln\frac{e^{1/4}}{\delta}}+b^1}{\alpha_i}
+\tfrac{4L}{3}\cdot \tfrac{\sigma^0\sqrt{\ln\frac{2}{\delta}}+b^0}{\alpha_i^2}
\right)
+\frac{L}{\eta}\left(1+\tfrac{4}{3}\left(m-|\mathbf{B}_{\eta}|\right)\right)
} \tag{23}
$$

適切に
$$
\sigma^0(n),\ \sigma^1(n),\ b^0(H),\ b^1(H)
$$
を十分小さく選ぶと、
$$
\mathbb{P}\left(|D_2| \le \frac{L(1+\frac{4m}{3})}{\eta}\right) \ge 1-\delta.
$$
これと $|D_1|\ge \frac{L(1+\frac{4m}{3})}{\eta}$を合わせて
$$
\mathbb{P}\left(|D_2|\le |D_1|\right) \ge 1-\delta.
$$

(16) に$|D_1|^2-|D_2|^2\ge 0$を代入し、
$$
\eta\sum_{i\in\mathbf{B}_{\eta}}\log \alpha_i(t+1) \ge \eta\sum_{i\in\mathbf{B}_{\eta}}\log \alpha_i(t).
$$
指数をとって
$$
\prod_{i\in\mathbf{B}_{\eta}(\theta_t)}\alpha_i(t+1) \ge \prod_{i\in\mathbf{B}_{\eta}(\theta_t)}\alpha_i(t)
\quad\text{（確率少なくとも }1-\delta\text{）}.
$$
さらに、ステップサイズの選択（後述の ($\gamma_t$) の下界）と滑らかさから各制約値は 1 ステップで高確率に ($\alpha_i(t+1)\ge \frac{1}{2}\alpha_i(t)$) を満たします。この 2つで「継続的な減少が抑止」されます。これで補題 D.5 が示されました。

### 補題 D.4 の証明

やることは 3 点：
(1) ($\mathbb{P}(\mathcal{A})\ge 1-\delta$)
(2) ($\mathbb{P}(\mathcal{B})\ge 1-\delta$)
(3) ($\mathbb{P}(\mathcal{C})\ge 1-\delta$)
最後に合成で ($\mathbb{P}(\mathcal{A}\cap\mathcal{B}\cap\mathcal{C})\ge 1-mT\delta$)を示すことです。

#### (1) 事象 ($\mathcal{A}$)：全反復で$\min_i V_i^{\theta_t}\ge c\eta$

*   ($\eta\le \nu_{\rm emf}$) を選び、補題 D.5 の条件を満たすように
    $$
    \sigma^{0,1}(n),\ b^{0,1}(H)
    $$
    を（質問文記載の形で）設定する。すると補題 D.5 が使える。
*   さらに Usmanova et al. (2024) Lemma 6 を組合せることで、
    $$
    \boxed{
    \mathbb{P}\left\{\forall t\le T: \min_i\alpha_i(t)\ge c\eta, \ \min_i\hat\alpha_i(t)\ge \tfrac{3}{8}c\eta\right\} \ge 1-mT\delta
    }
    $$
    が得られる（式 (24)）。ここで
    ($c=\left(\frac{\ell(1-\gamma)^2}{4M_g(1+\frac{4m}{3})}\right)^m$)。

これにより ($\mathcal{A}$)（かつ ($\hat\alpha$) の下限）を高確率で確保。

---

#### (2) 事象 ($\mathcal{B}$)：$\gamma_t\ge C\eta$

($\hat\alpha_i(t)$) の下限（上の (1)）を ($\gamma_t$) の構成（Sec. D.3.1）に代入すると、
（Usmanova et al. 2024, Lemma 6）より
$$
\boxed{ \gamma_t \ge C\eta } \quad\text{（式 (25)）}
$$
が得られる。ここで ($C=\frac{c}{2 L^{2}\left(1+\frac{m}{c}\right) \max \left\{4+\frac{5 M c}{L^{2}}, 1+\sqrt{\frac{M c}{4 L^{2}}}\right\}}$)。
従って ($\mathbb{P}(\mathcal{B})\ge 1-\delta$)。

---

#### (3) 事象 ($\mathcal{C}$)：$|\Delta_t|\ge \eta/4$

命題 3.3 を、(1) で得た ($\alpha,\hat\alpha$) の下限に従って最適に噛み合わせるため
$$
\begin{aligned}
\sigma^0(n)&\le \min\left\{\frac{\alpha_i(t)\min\{2\alpha_i(t),\eta\}}{8\eta\sqrt{\ln\frac{2}{\delta}}},
\frac{1}{\left(\sum_{i=1}^m \tfrac{16L}{\alpha_i(t)\hat\alpha_i(t)}\right)\sqrt{\ln\frac{2}{\delta}}}\right\}, \\
\sigma^1(n)&\le \min\left\{\frac{L\alpha_i(t)}{3\eta\sqrt{\ln\frac{e^{1/4}}{\delta}}},
\frac{\eta}{16\left(1+\sum_{i=1}^m \tfrac{\eta}{\hat\alpha_i(t)}\right)\sqrt{\ln\frac{e^{1/4}}{\delta}}}\right\},
\end{aligned}
$$
同様に ($b^0(H),b^1(H)$) を（質問文の式 (26),(27)）のように選ぶと、
$$
\boxed{ \mathbb{P}\left(|\Delta_t|\ge \tfrac{\eta}{4}\right) \ge 1-\delta } \tag{28}
$$
が得られます。

(1)–(3) を合成し、合併界で
$$
\boxed{ \mathbb{P}\{\mathcal{A}\cap\mathcal{B}\cap\mathcal{C}\} \ge 1-mT\delta }.
$$
最後に、(26)–(27) を満たすための
$$
n = \mathcal{O}\left(\frac{\ln\frac{1}{\delta}}{(1-\gamma)^{6+8m} \ell^{4m} \eta^{4}}\right),\qquad
H = \mathcal{O}\left(\log\frac{1}{(1-\gamma)^{3+4m} \ell^{2m} \eta^{2}}\right)
$$
を選べば、命題の証明が完了でうす。お疲れ様でした。
